In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Create a phonebook

A phonebook maps clusters to partitions under the following constraints.

1. All clusters in the same bucket MUST go in the same partition.
2. A partition CANNOT contain clusters totalling more than N records.
3. Clusters that are similar to each other SHOULD go in the same partition.

## Aproach

1. Choose a maximum number of partitions and a maximum number of records per partition.
2. Compute the top N most-similar clusters to each cluster.
3. Start by putting the clusters for each bucket in their own partition.
4. Some buckets because of their size will span multiple partitions. Split those buckets up-front.
5. While there are more than the maximum number of partitions, merge partitions containing the most-similar (to least-similar) clusters so long as the combined partition doesn't exceed the maximum partition size.
6. If you end up with too many partitions, or many partitions that have significantly fewer number of records than the maximum, consider repeating the steps with a lower maximum number of records per partition.

In [ ]:
import json
import re

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

from src.data.utils import read_csv
from src.models.utils import top_similar_names

In [ ]:
# configure
given_surname = "given"

n_partitions = 720
max_partition_size = 2_100_000
min_threshold = 0.6

linkage = "average"
similarity_threshold = 0.1
scorer = "ce"
cluster_freq_normalizer = "none"
clusters_path = f"../data/processed/clusters_{given_surname}-{scorer}-{linkage}-{similarity_threshold}-{cluster_freq_normalizer}-augmented.json"
super_clusters_path = f"../data/processed/super_clusters_{given_surname}-{scorer}-{linkage}-{similarity_threshold}-{cluster_freq_normalizer}.json"
pref_path = f"s3://familysearch-names/processed/tree-preferred-{given_surname}-aggr.csv.gz"

phonebook_path = f"s3://familysearch-names/processed/phonebook.csv"

### Load data

In [ ]:
# dict of name -> freq
pref_df = read_csv(pref_path)
name_freq = {name: freq for name, freq in zip(pref_df['name'], pref_df['frequency']) \
                if len(name) > 1 and re.fullmatch(r'[a-z]+', name)}
pref_df = None

In [ ]:
print(sum(list(name_freq.values())))

In [ ]:
# load clusters and super-clusters
name_cluster = {}       # name -> cluster position
cluster_centroids = []  # centroid for each cluster
cluster_labels = []     # label for each cluster
cluster_indexes = {}    # cluster label -> index
cluster_sizes = []      # number of records for each cluster

with open(clusters_path, 'r') as f:
    clusters = json.load(f)  # cluster label -> names, centroid

with open(super_clusters_path, 'r') as f:
    super_clusters = json.load(f)  # super_cluster label -> cluster labels

for label, cluster in clusters.items():
    n_records = 0
    for name in cluster['names']:
        name_cluster[name] = len(cluster_labels)
        n_records += name_freq.get(name, 0)
    cluster_indexes[label] = len(cluster_labels)
    cluster_labels.append(label)
    cluster_sizes.append(n_records)
    cluster_centroids.append(np.array(cluster['centroid']))
cluster_labels = np.array(cluster_labels)

In [ ]:
print('number of names', len(name_cluster))
print('number of clusters', len(cluster_labels))
print('number of super-clusters', len(super_clusters))

In [ ]:
print('size of all clusters', sum(cluster_sizes))

In [ ]:
print('perfect cluster size', sum(cluster_sizes) / n_partitions)

## Compute nearby clusters

In [ ]:
cluster_similarity_scores = []  # [(ix1, ix2, similarity)]
top_n = 1000
for ix, centroid in tqdm(enumerate(cluster_centroids), mininterval=2.0):
    nearby_clusters, similarities = top_similar_names(centroid, cluster_centroids, cluster_labels,
                                                      threshold=min_threshold, top_n=top_n)
    for nearby_cluster, similarity in zip(nearby_clusters, similarities):
        nearby_ix = cluster_indexes[nearby_cluster]
        if ix >= nearby_ix:
            continue
        cluster_similarity_scores.append((ix, nearby_ix, similarity))

In [ ]:
cluster_similarity_scores = sorted(cluster_similarity_scores, key=lambda x: x[2], reverse=True)

In [ ]:
len(cluster_similarity_scores)

## Create phonebook

In [ ]:
partition_ix = 0
partition_clusters = {}  # partition -> [cluster ix]
partition_size = {}      # partition -> size
cluster_partition = {}   # cluster -> partition ix

In [ ]:
def merge_partitions(ix1, ix2):
    global partition_ix, partition_clusters, partition_size, cluster_partition
    
    partition_clusters[partition_ix] = partition_clusters[ix1] + partition_clusters[ix2]
    del partition_clusters[ix1]
    del partition_clusters[ix2]
    partition_size[partition_ix] = partition_size[ix1] + partition_size[ix2]
    del partition_size[ix1]
    del partition_size[ix2]
    for cluster in partition_clusters[partition_ix]:
        cluster_partition[cluster] = partition_ix
    partition_ix += 1

### Start by creating a partition for each cluster

In [ ]:
for ix in range(len(cluster_labels)):
    partition_clusters[partition_ix] = [ix]
    partition_size[partition_ix] = cluster_sizes[ix]
    cluster_partition[ix] = partition_ix
    partition_ix += 1

In [ ]:
partition_ix

### Merge clusters for the same super-cluster

In [ ]:
for super_cluster in super_clusters.values():
    first_cluster_label = super_cluster[0]
    for cluster_label in super_cluster[1:]:
        ix1 = cluster_partition[cluster_indexes[first_cluster_label]]
        ix2 = cluster_partition[cluster_indexes[cluster_label]]
        if ix1 != ix2:
            merge_partitions(ix1, ix2)

In [ ]:
len(partition_clusters)

### Count partitions that are > max size

In [ ]:
n_extra_partitions = 0
remaining_total = 0
for ix, size in partition_size.items():
    while size > max_partition_size:
        print(size, ",".join(cluster_labels[cix] for cix in partition_clusters[ix]))
        n_extra_partitions += 1
        size -= max_partition_size
    remaining_total += size

In [ ]:
print('extra', n_extra_partitions, 
      'remaining', n_partitions - n_extra_partitions, 
      'remaining size', remaining_total, 
      'remaining/partition', remaining_total / (n_partitions - n_extra_partitions))

### Merge partitions containing most-similar clusters

In [ ]:
for cix1, cix2, _ in tqdm(cluster_similarity_scores, mininterval=2.0):
    if len(partition_clusters) <= (n_partitions - n_extra_partitions):
        break
    pix1 = cluster_partition[cix1]
    pix2 = cluster_partition[cix2]
    if pix1 != pix2 and partition_size[pix1] + partition_size[pix2] <= max_partition_size:
        merge_partitions(pix1, pix2)    

In [ ]:
len(partition_clusters)

### Merge other partitions using first-fit

In [ ]:
while len(partition_clusters) > (n_partitions - n_extra_partitions):
    # find the smallest partition
    small_ix = min(partition_size, key=partition_size.get)
    # find the largest partition it will fit into
    large_ix = None
    for pix, size in partition_size.items():
        if pix == small_ix or partition_size[small_ix] + size > max_partition_size:
            continue
        if large_ix is None or size > partition_size[large_ix]:
            large_ix = pix
    if large_ix is None:
        break
    # merge them
    merge_partitions(small_ix, large_ix)

In [ ]:
len(partition_clusters)

### Check partition sizes

In [ ]:
if len(partition_clusters) > (n_partitions - n_extra_partitions):
    print('We have a problem!')
len(partition_clusters)

In [ ]:
# count number of partitions that are less than 90% full, 75% full, half full
for threshold in [0.90, 0.75, 0.5]:
    print(threshold, len([size for size in partition_size.values() if size < max_partition_size * threshold]))

## Plot partition sizes

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist([size for size in partition_size.values() if size <= max_partition_size], bins=100)
plt.tight_layout()
plt.show()

## Plot scores

In [ ]:
same_partition_scores = []
diff_partition_scores = []
for cix1, cix2, score in cluster_similarity_scores:
    if cix1 == cix2:
        continue
    pix1 = cluster_partition[cix1]
    pix2 = cluster_partition[cix2]
    if pix1 == pix2:
        same_partition_scores.append(score)
    else:
        diff_partition_scores.append(score)
print(len(same_partition_scores))
print(len(diff_partition_scores))

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(same_partition_scores, bins=100, alpha=0.5, label="same", color='green')
plt.hist(diff_partition_scores, bins=100, alpha=0.5, label="diff", color='red')
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

## Save phonebook